In [ ]:
from glass.ete.otol.pg import osm_to_lulc
from glass.prop.prj import get_epsg
import datetime
import os

osm = '/mnt/d/osm2lulc02/osmdata/osm_terceira.xml'

nomenclature = 'clc'

refraster = '/mnt/d/osm2lulc02/lmt/rst_terceira.tif'

result = '/mnt/d/osm2lulc02/lulc_fmosm/terceira.gpkg'

db = '/mnt/d/osm2lulc02/lulc_fmosm/terceira_26062023.sql'

ws = '/mnt/d/osm2lulc02/debug'

time_a = datetime.datetime.now().replace(microsecond=0)

res, logs = osm_to_lulc(
    osm, nomenclature, get_epsg(refraster),
    result, savedb=db, tmpfld=ws
)

time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
shp = '/mnt/d/osm2lulc02/lulc_fmosm/terceira.gpkg'

lyr = 'osmtolulc_v2'

col = 'lulc'

osm_db = 'osm_terceira'

In [ ]:
from glass.ete.otol.tools import classes_priority
from glass.sql.q import q_to_obj
import datetime

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

dif = q_to_obj(osm_db, (
    "SELECT tbla.fid AS fid_a, tbla.geom AS geom_a, "
    "tblb.fid AS fid_b, tblb.geom AS geom_b "
    "FROM tblcls_28 AS tbla "
    "LEFT JOIN tblcls_86 AS tblb "
    "ON ST_Intersects(tbla.geom, tblb.geom)"
))

time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

dif = q_to_obj(osm_db, (
    "SELECT tbla.fid AS fid_a, tbla.geom AS geom_a, "
    "tblb.fid AS fid_b, tblb.geom AS geom_b, "
    "CASE "
    	"WHEN tblb.fid IS NOT NULL THEN "
    	"ST_Difference(tbla.geom, tblb.geom) "
    	"ELSE NULL "
    "END AS difgeom "
    "FROM tblcls_28 AS tbla "
    "LEFT JOIN tblcls_86 AS tblb "
    "ON ST_Intersects(tbla.geom, tblb.geom)"
))

time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
order_cls = classes_priority(2)

In [ ]:
order_cls

In [ ]:
from glass.it.db import gpkg_lyr_attr_to_psql

In [ ]:
gpkg_lyr_attr_to_psql(shp, lyr, col, osm_db, 'tblcls')

In [ ]:
from glass.ete.otol.tools import module_osmtags

tags = module_osmtags(2)

In [ ]:
tags[tags.name == 'roads'].lulc_id.unique()

In [ ]:
tags.name.unique()

In [ ]:
tags[
    (tags['value'] == 'yes') &
    (tags['key'] == 'building')
]

In [ ]:
from glass.ete.osm2lulc.grs import osm_to_lulc
from glass.ete.osm2lulc.utils import record_time_consumed
import datetime
import os
from glass.pys.oss import fprop, mkdir
from glass.pys.tm import now_as_str

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

ws = mkdir(os.path.join(
    os.path.dirname(osm),
    fprop(result, 'fn')
), overwrite=True)

shp, time_check = osm_to_lulc(
    osm, nomenclature, refraster,
    result, tmpfld=ws,
    overwrite=True, savedb=db
)

time_b = datetime.datetime.now().replace(microsecond=0)
    
# Record time consumed in xlsx table
record_time_consumed(time_check, os.path.join(
    os.path.dirname(result),
    fprop(result, 'fn') + '.xlsx'
))

print(time_b - time_a)